<img style="float: right; padding: 10px;" width="200" src="images/logo-onem2m-the-iot-standard.jpg">


# oneM2M - Access Control

This notebook demonstrates how access control to resources can be done in oneM2M.

- Create an &lt;ACP> resource with different credentials for a new originator
- Create a second &lt;AE> resource with the new access controls policy
- Succeed to add a &lt;container> to the second &lg;AE> resource
- Fail to update the second &lt;AE> resource
 
 
<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 550px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
    In the previous notebooks we always used the originator <i>Cmyself</i> to create resources under the &lt;AE> resource for that originator. This succeedded because, if no other restrictions apply, the originator can always access the resources under its own &lt;AE> resource. 
<br><br>
This notebook gives a short overview about how access to resources can be restricted and controlled.
    </div>
</div>
</p>
 


## Intitialization
The section does import necessary modules and configurations, and prepares the CSE for this notebook.

<p>
<div  style="background-color:transparent;border:2px #005480 solid;padding: 10px;width: calc(100% - 550px);border-radius:10px;">
    <div style="color:#005480;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Note</b></div>
    <div style="color:#545054;">
        This will create the same structure for a street light scenario as for the <a href="04-groups.ipynb" target="_new">Group notebook</a>. 
    </div>
</div>
</p>

In [ ]:
%run src/init.py acp

## Create \<ContentInstance> Resources for each Street Light

<img align="right" width="500px" src="images/05/create_contentInstance_1.png">

This section has two requests. 

First we will create a \<contentInstance> under the *StreetLight-AE-1/Light-Container-1* structure.

<p>
<div  style="background-color:transparent;border:2px #005480 solid;padding: 10px;width: calc(100% - 550px);border-radius:10px;">
    <div style="color:#005480;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Note</b></div>
    <div style="color:#545054;">
         We will use the same originator <i>CstreetLight-AE-1</i> for both requests.
    </div>
</div>
</p>



In [ ]:
CREATE (                                             # CREATE request
    
    # Send the request to the <fopt> virtual resoure
    to                      = cseBaseName + '/StreetLight-AE-1/Light-Container-1',
    
    # Request Parameters
    originator              = 'CstreetLight-AE-1',   # Set the originator
    requestIdentifier       = '123',                 # Unique request identifier
    releaseVersionIndicator = '3',                   # Release version indicator
    resourceType            = Type.ContentInstance,  # Type of the resource
    
    # Request Body
    primitiveContent = 
        {
            'm2m:ContentInstance': {
                'contentInfo': 'text/plain:0',
                'content': 'lights-on'
            }
        }
)

<img align="right" width="400px" src="images/05/create_contentInstance_2_fail.png">


This request should have been processed successfully.

The second request will try to create a \<contentInstance> under the *StreetLight-AE-2/Light-Container-2* structure with the same originator *CstreetLight-AE-1*.

In [ ]:
CREATE (                                             # CREATE request
    
    # Send the request to the <fopt> virtual resoure
    to                      = cseBaseName + '/StreetLight-AE-2/Light-Container-2',
    
    # Request Parameters
    originator              = 'CstreetLight-AE-1',   # Set the originator
    requestIdentifier       = '123',                 # Unique request identifier
    releaseVersionIndicator = '3',                   # Release version indicator
    resourceType            = Type.ContentInstance,  # Type of the resource
    
    # Request Body
    primitiveContent = 
        {
            'm2m:ContentInstance': {
                'contentInfo': 'text/plain:0',
                'content': 'lights-on'
            }
        }
)

**This request should have failed with an error that the originator hasn't the appropriate access to the resource.**

<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 550px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
    By default in oneM2M only the creator of a resource has access to that resource.
    </div>
</div>
</p>
 


Let's fix this by giving the originator *CstreetLight-AE-1* access to the *StreetLight-AE-2/Light-Container-2* resource.

## Create an \<ACP> Resource

<img align="right" width="500px" src="images/05/create_acp.png">


Access Control Policies are used to associate access control with identities. They define the rules to for access control to resources. Each &lt;ACP> resource has two sections:

- **pv (Privileges)** : The actual privileges defined by this policy.
- **pvs (Self-Privileges)** : This defines the privileges necessary to access and control the &lt;ACP> resource itself.

Each section defines, at least, the request originators and the allowed operations. Other parameters here could be date and time ranges, geo-location, network addresses, and more.

The privileges include an *operations* field. Here, the various request operations, for example CREATE or RETRIEVE, are expressed as a single number that represents a bitfield as shown in the following table:

        
| Value | Operation      |
|:------|:---------------|
| 1     | CREATE         |
| 2     | RETRIEVE       |
| 4     | UPDATE         |
| 8     | DELETE         |
| 16    | NOTIFY         |
| 32    | DISCOVERY      |



The first step is to create an \<ACP> resource that defined the privileges that will give the originator *CstreetLight-AE-1* CREATE access the *StreetLight-AE-2/Light-Container-2* resource. 
After that the \<ACP> resource needs to be assigned to the resource that should follow these new accss policies.The first step is to create an \<ACP> resource that defined the *Access Control Privileges* that will give the originator *CstreetLight-AE-1* CREATE access the *StreetLight-AE-2/Light-Container-2* resource.


<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 550px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
Technically, &lt;ACP> resource can be located directly under the &lt;CSEBase> or under an &lt;AE> resource. We will create the &lt;ACP> resource under the &lt;AE> resource where the policies apply.  
This makes it a little bit easier to manage, e.g. when the &lt;AE> resource is deleted the &lt;ACP> resource is removed as well.
<br><br>
Note, that in the following request we need to add a policy for the original creator as well because the default rule (ie. the original creator has full access) doesn't apply when access control policies are assigned.
    </div>
</div>
</p>
 

In [ ]:

CREATE (                                                                   # CREATE request
    
    # Create the ACP resource under the CSEBase
    to                      = cseBaseName + '/StreetLight-AE-2',  
    
    # Request Parameters
    originator              = 'CstreetLight-AE-2',                         # Assign an originator ID, must start with 'C'
    requestIdentifier       = '123',                                       # Unique request identifier
    releaseVersionIndicator = '3',                                         # Release version indicator
    resourceType            = Type.ACP,                                    # Type of the resource: Access Control Policy
    
    # Request Body
    primitiveContent = 
        {
            'm2m:AccessControlPolicy': {
                'resourceName' : 'Streetlight-ACP-2',                      # Name of the resource 
                'privileges' : {                                           # Privileges used in access to other resources
                    'accessControlRule': [ {                               # List of access control rules
                        'accessControlOriginators': ['CstreetLight-AE-1'], # Access control originator
                        'accessControlOperations': 1                       # Access control operations (bitfield): only CREATE
                    },
                    {                                                      # List of access control rules
                        'accessControlOriginators': ['CstreetLight-AE-2'], # Access control originator
                        'accessControlOperations': 63                      # Access control operations (bitfield): All
                    }]
                },
                'selfPrivileges': {                                        # Privileges used in access to this <ACP> resource
                    'accessControlRule' : [{                               # List of access control rules
                        'accessControlOriginators': ['CstreetLight-AE-2'], # Access control originator
                        'accessControlOperations': 63                      # Access control operations (bit-field): ALL
                    }]
                }
            }
        }

)

### Requests & Response Attributes

The request and response introduce new attributes:

| Attribute                | Short Name | Description                                                                                                                               |
|:-------------------------|:-----------|:------------------------------------------------------------------------------------------------------------------------------------------|
| privileges               | pv         | The privileges for other originators defined by this access control policy.                                                               |
| selfPrivileges           | pvs        | The privileges necessary to access and control this &lt;ACP> resource itself.                                                             |
| accessControlRule        | acr        | A set of control rules for the privilegs                                                                                                  |
| accessControlOriginators | acor       | This list includes the *originator* information. The parameter comprises a list of domain, CSE-IDs, AE-IDs, and other oneM2M identifiers. |
| accessControlOperations  | acop       | This number represents a bit-field of granted privileges. The following table shows the mapping.                                          |


## Assign the \<ACP> Resource

<img align="right" width="500px" src="images/05/update_container.png">



For the second step to give *CstreetLight-AE-1* access to *CstreetLight-AE-2's* resource we need to update the \<Container> resource and assigns the just created \<ACP> resource to it.  
This is done with the following request where we update the \<Container> resource's *accessControlPolicyIDs* attribute.

In [ ]:
UPDATE (                                        # UPDATE request
    
    # Update the <container>
    to                      = cseBaseName + '/StreetLight-AE-2/Light-Container-2',  
    
    # Request Parameters
    originator              = 'CstreetLight-AE-2',         # Set the originator
    requestIdentifier       = '123',            # Unique request identifier
    releaseVersionIndicator = '3',              # Release version indicator
    
    # Request Body
    primitiveContent = 
        {
            'm2m:Container' : {
                'accessControlPolicyIDs': [ cseBaseName + '/StreetLight-AE-2/Streetlight-ACP-2' ]
            }
        }
)

The *accessControlPolicyIDs* attribute now contains a reference to the \<ACP> resource. The CSE will take the access control policies defined there into account when checking access rights.


<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 550px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
        A resource may have more than one refeences assigned to its <i>accessControlPolicyIDs</i> attribute. To get access to a resource it is sufficient that only one of the referenced policies applies.
        
Also note, that the &lt;ACP>'s resource identifier in the <i>accessControlPolicyIDs</i> attribute has been changed to unstructured format.
    </div>
</div>
</p>

<img align="right" width="500px" src="images/05/create_contentInstance_2.png">



## Create a \<ContentInstance> Resource for the second Street Light



With the next request we try again to create a /<ContentInstance> resource under the *StreetLight-AE-2/Light-Container-2* \<Container> resource. This time this request should succeed.
 

In [ ]:
CREATE (                                             # CREATE request
    
    # Send the request to the <fopt> virtual resoure
    to                      = cseBaseName + '/StreetLight-AE-2/Light-Container-2',
    
    # Request Parameters
    originator              = 'CstreetLight-AE-1',   # Set the originator
    requestIdentifier       = '123',                 # Unique request identifier
    releaseVersionIndicator = '3',                   # Release version indicator
    resourceType            = Type.ContentInstance,  # Type of the resource
    
    # Request Body
    primitiveContent = 
        {
            'm2m:ContentInstance': {
                'contentInfo': 'text/plain:0',
                'content': 'lights-on'
            }
        }
)

Now, the second request should have succeeded. The originator *CstreetLight-AE-1* is able to create new \<contentInstance> resources under the \<container> resource *Light-Container-2*.